## Overview

The EMISOpen consultation writeback is essentially the same as HL7 v2 ORU Unsolicited Observation.

| HL7 v2.4 Segment                                                     | HL7 FHIR Resource                                                                                           | EMISOpen                             |
|----------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|--------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PID)   | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)            | RegistrationType (extends IdentType) |
| [PV1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PV1})  | [Encounter](https://virtually-healthcare.github.io/R4/StructureDefinition-Encounter.html)                   | ConsultationType                     |
| - [OBX](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/OBX) | [Observation](https://virtually-healthcare.github.io/R4/StructureDefinition-Observation.html)               | EventType (both coded and uncoded)   |
| - [AL1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/AL1) | [AllergyIntolerance](https://virtually-healthcare.github.io/R4/StructureDefinition-AllergyIntolerance.html) | AllergyType                          |
| - [DG1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/DG1) | [Condition](https://virtually-healthcare.github.io/R4/StructureDefinition-Condition.html)                   | EventType (with problem elements)    |
| - [PR1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PR1) | [Procedure](https://virtually-healthcare.github.io/R4/StructureDefinition-Procedure.html) |